**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip install sk-video 
import skvideo.io
import cv2
import json
import os 

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


    100% |████████████████████████████████| 2.3MB 11.1MB/s 


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act predicts action to be taken at time $t+1$ in a given state $s$ at a time $t$ and chooses between exploration (with a probability epsilon) and exploitation (with a probability 1-epsilon). 

More specifically, during the training, the action can be chosen at random for exploration or can be chosen according to the knowledge of the network. The epsilon is essential for the exploration part of the RL. During the test however, only the exploration phase is chosen.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    directory = os.path.join(os.getcwd(), name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position localizes the rat with a white pixel (value of 1), the background of the grid with black pixel (value of -1) and the rewards in the grid (value of 0.5 for food and -1 for poison). Therefore, the array encodes information about positions of rewards and the rat with  pixel's color. 

The array board has the reward associated to each position. Once a position is visited the reward is set to zero for this position.

Both arrays have the same squarred shape of size grid_size.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
 
    for e in range(epochs):
        
        win = 0
        lose = 0
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        game_over = False

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.5/6.0. Average score (1.5)
Win/lose count 9.5/15.0. Average score (-2.0)
Win/lose count 8.0/14.0. Average score (-3.3333333333333335)
Win/lose count 8.5/10.0. Average score (-2.875)
Win/lose count 11.0/11.0. Average score (-2.3)
Win/lose count 8.5/15.0. Average score (-3.0)
Win/lose count 9.0/14.0. Average score (-3.2857142857142856)
Win/lose count 11.0/12.0. Average score (-3.0)
Win/lose count 11.0/16.0. Average score (-3.2222222222222223)
Win/lose count 4.5/16.0. Average score (-4.05)
Final score: -4.05


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**

a) By definition, $Q^{\pi}(s,a) = E_{\pi}[R_t |s_t = s, a_t = a]$. Therefore, we can get the following:

\begin{align}
 Q^{\pi}(s,a) &=E_\pi[\sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\ &= E_\pi[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\ &= E_\pi[r(s,a) + \gamma \sum_{p=0}^{\infty} \gamma^{p} r_{t+1+p}|s_t = s, a_t = a]
 \\ &= E_\pi[r(s,a) + \gamma R_{t+1}|s_t = s, a_t = a]
\end{align}


Using the law of total expectancy $\mathbb E[X]=\mathbb E[\mathbb E[X|Y]]$ we then have : 

\begin{align}
  Q^{\pi}(s,a) &= E_\pi[r(s,a) + \gamma Q^{\pi}(s_{t+1},a_{t+1}) |s_t = s, a_t = a] \\ 
  &=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

b) By definition, we have:

\begin{align} 
Q^*(s,a) = \max_{\pi} Q^{\pi}(s, a) 
\end{align}

and by definition of policy we have $\pi(.) = \pi^*(. |s)$, hence: 

\begin{align} 
Q^*(s,a) &=E_{\pi^*(. |s)}[r(s,a)+\gamma \max_{\pi} Q^{\pi}(s',a')] \\
&=E_{\pi^*(. |s)}[r(s,a)+\gamma \max_{a'} Q^{\pi^*}(s',a')] 
\end{align}

c) A plausible objective could be based on MSE:

\begin{align} 
\mathcal{L}(\theta) = E_{\pi^*(. |s)}[\Vert Q^*(s,a)- Q(s, a; \theta)\Vert^2] 
\end{align}

Using the answer at b), we get 

\begin{align} 
\mathcal{L}(\theta) = E_{\pi^*(. |s)}[\Vert r(s,a) +\gamma\max_{a'}Q(s',a',\theta)- Q(s, a; \theta)\Vert^2] 
\end{align}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append(m)

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0)), axis=1).astype(int)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # Sample a minibatch from the replay memory
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i]  = self.model.predict(np.expand_dims(n_s_, axis=0))
         
            if game_over_:
                target_q[i,a_] = r_ 
            else:
                target_q[i,a_] = r_ + self.discount * np.amax(self.model.predict(np.expand_dims(n_s_, axis=0)), axis=1)
              

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(64, activation='relu', input_dim=5*5*self.n_state))
        model.add(Dense(32, activation='relu',))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/100 | Loss 0.0419 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/100 | Loss 0.0116 | Win/lose count 5.5/7.0 (-1.5)
Epoch 002/100 | Loss 0.0056 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/100 | Loss 0.0177 | Win/lose count 5.0/2.0 (3.0)
Epoch 004/100 | Loss 0.0109 | Win/lose count 5.0/6.0 (-1.0)
Epoch 005/100 | Loss 0.0096 | Win/lose count 3.0/8.0 (-5.0)
Epoch 006/100 | Loss 0.0104 | Win/lose count 3.0/6.0 (-3.0)
Epoch 007/100 | Loss 0.0164 | Win/lose count 2.5/5.0 (-2.5)
Epoch 008/100 | Loss 0.0705 | Win/lose count 3.0/4.0 (-1.0)
Epoch 009/100 | Loss 0.0136 | Win/lose count 5.0/6.0 (-1.0)
Epoch 010/100 | Loss 0.0672 | Win/lose count 8.5/6.0 (2.5)
Epoch 011/100 | Loss 0.0042 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/100 | Loss 0.0173 | Win/lose count 9.5/9.0 (0.5)
Epoch 013/100 | Loss 0.0652 | Win/lose count 10.0/7.0 (3.0)
Epoch 014/100 | Loss 0.0167 | Win/lose count 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
from keras.layers import Conv2D

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### NN Model
        model = Sequential()
        model.add(Conv2D(32, 2, activation='relu', input_shape=(5,5,self.n_state)))
        model.add(Conv2D(32, 2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/100 | Loss 0.0034 | Win/lose count 2.0/6.0 (-4.0)
Epoch 001/100 | Loss 0.0052 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/100 | Loss 0.0087 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/100 | Loss 0.0163 | Win/lose count 7.0/6.0 (1.0)
Epoch 004/100 | Loss 0.0039 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/100 | Loss 0.0136 | Win/lose count 7.0/4.0 (3.0)
Epoch 006/100 | Loss 0.0023 | Win/lose count 6.0/5.0 (1.0)
Epoch 007/100 | Loss 0.0024 | Win/lose count 5.0/4.0 (1.0)
Epoch 008/100 | Loss 0.0525 | Win/lose count 12.5/4.0 (8.5)
Epoch 009/100 | Loss 0.0128 | Win/lose count 10.5/3.0 (7.5)
Epoch 010/100 | Loss 0.0076 | Win/lose count 2.0/2.0 (0.0)
Epoch 011/100 | Loss 0.0146 | Win/lose count 10.0/1.0 (9.0)
Epoch 012/100 | Loss 0.0047 | Win/lose count 6.0/0 (6.0)
Epoch 013/100 | Loss 0.0020 | Win/lose count 7.5/2.0 (5.5)
Epoch 014/100 | Loss 0.0019 | Win/lose count 9.0/5.0 (4.0)
Epoch 015/100 | Loss 0.0018 | Win/lose count 8.5/6.0 (2.5)
Epoch 016/100 | Loss 0.0034 | Win/lose count 12.5/4.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.5/1.0. Average score (2.5)
Win/lose count 11.5/2.0. Average score (6.0)
Win/lose count 6.5/0. Average score (6.166666666666667)
Win/lose count 5.0/0. Average score (5.875)
Win/lose count 7.0/0. Average score (6.1)
Win/lose count 7.5/1.0. Average score (6.166666666666667)
Win/lose count 3.0/2.0. Average score (5.428571428571429)
Win/lose count 4.0/1.0. Average score (5.125)
Win/lose count 8.5/1.0. Average score (5.388888888888889)
Win/lose count 5.5/1.0. Average score (5.3)
Final score: 5.3
Test of the FC
Win/lose count 4.5/3.0. Average score (1.5)
Win/lose count 5.5/1.0. Average score (3.0)
Win/lose count 3.5/2.0. Average score (2.5)
Win/lose count 4.0/1.0. Average score (2.625)
Win/lose count 3.0/1.0. Average score (2.5)
Win/lose count 5.5/0. Average score (3.0)
Win/lose count 9.0/2.0. Average score (3.5714285714285716)
Win/lose count 1.0/0. Average score (3.25)
Win/lose count 5.0/0. Average score (3.4444444444444446)
Win/lose count 2.5/1.0. Average sc

In [16]:
HTML(display_videos('cnn_test0.mp4'))

In [17]:
HTML(display_videos('fc_test0.mp4'))


The temperature increases the density of reward pixel and therefore improves perfomance of the model (less sparse reward). However, we note that the rat doesn't explore the game and tends to oscillate inside a square.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    epsilon_start = 1.0
    epsilon_end = 0.1
    
    # The epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epoch)

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        agent.set_epsilon(epsilons[e])
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #NEW
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        #new
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]   
        
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time 
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 
        
        self.board = bonus + malus 
        
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error, 2 times same line
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore0.mp4'))

Epoch 000/100 | Loss 0.0408 | Win/lose count 12.0/48.60000000000005 (-36.60000000000005)
Epoch 001/100 | Loss 0.0317 | Win/lose count 11.5/46.599999999999994 (-35.099999999999994)
Epoch 002/100 | Loss 0.0285 | Win/lose count 10.5/48.70000000000007 (-38.20000000000007)
Epoch 003/100 | Loss 0.0936 | Win/lose count 13.5/49.60000000000004 (-36.10000000000004)
Epoch 004/100 | Loss 0.0250 | Win/lose count 15.5/49.90000000000002 (-34.40000000000002)
Epoch 005/100 | Loss 0.0301 | Win/lose count 11.5/56.00000000000002 (-44.50000000000002)
Epoch 006/100 | Loss 0.0328 | Win/lose count 12.5/50.40000000000005 (-37.90000000000005)
Epoch 007/100 | Loss 0.0187 | Win/lose count 12.5/42.20000000000006 (-29.70000000000006)
Epoch 008/100 | Loss 0.0270 | Win/lose count 7.5/48.60000000000002 (-41.10000000000002)
Epoch 009/100 | Loss 0.0891 | Win/lose count 11.0/46.70000000000004 (-35.70000000000004)
Epoch 010/100 | Loss 0.0332 | Win/lose count 16.5/34.300000000000026 (-17.800000000000026)
Epoch 011/100 | Lo

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

We can see that the new classifier network learns very fast to mimick the expert DQN and its performance is better. However we should keep in mind that its performances is limited by it's unability to explore and learn new states. 

***

In [0]:
class Classifier(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Classifier, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,2,input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
    def learn(self, s_, a_):

        input_states = np.zeros((1, 5,5,self.n_state))
        target_q = np.zeros((1, 4))
        
        target_q[0, a_] = 1
        input_states[0] = s_
            

        l = self.model.train_on_batch(input_states, target_q)

        return l

def train_behavior(agent_ref, classifier,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent_ref.act(state, train=False)

            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Learn classifier
            loss = classifier.learn(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [22]:
classifier = Classifier(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_behavior(agent, classifier, env, 20, prefix='cnn_train_behavior_clone')

# Evaluation
test(classifier,env,epochs_test,prefix='cnn_test_behavior_clone')

Epoch 000/020 | Loss 0.2050 | Win/lose count 27.0/17.499999999999986 (9.500000000000014)
Epoch 001/020 | Loss 0.1559 | Win/lose count 24.5/13.499999999999977 (11.000000000000023)
Epoch 002/020 | Loss 0.0477 | Win/lose count 25.0/19.500000000000007 (5.499999999999993)
Epoch 003/020 | Loss 0.0925 | Win/lose count 20.5/21.599999999999998 (-1.0999999999999979)
Epoch 004/020 | Loss 0.1590 | Win/lose count 34.0/13.799999999999978 (20.200000000000024)
Epoch 005/020 | Loss 0.2340 | Win/lose count 22.5/14.29999999999998 (8.20000000000002)
Epoch 006/020 | Loss 0.1776 | Win/lose count 27.0/13.899999999999979 (13.100000000000021)
Epoch 007/020 | Loss 0.1345 | Win/lose count 25.5/19.79999999999999 (5.70000000000001)
Epoch 008/020 | Loss 0.1431 | Win/lose count 25.0/23.500000000000004 (1.4999999999999964)
Epoch 009/020 | Loss 0.1201 | Win/lose count 19.5/21.5 (-2.0)
Epoch 010/020 | Loss 0.1270 | Win/lose count 25.5/16.199999999999985 (9.300000000000015)
Epoch 011/020 | Loss 0.1895 | Win/lose count 2